In [14]:
import os
import base64
from mistralai import Mistral

def encode_pdf(pdf_path):
    """Encode the PDF to base64."""
    try:
        with open(pdf_path, "rb") as pdf_file:
            return base64.b64encode(pdf_file.read()).decode('utf-8')
    except FileNotFoundError:
        print(f"Error: The file {pdf_path} was not found.")
        return None
    except Exception as e:
        print(f"Error: {e}")
        return None

# Get API key
api_key = os.environ.get("OCR_MISTRAL")
if not api_key:
    print("Error: OCR_MISTRAL environment variable not found")
else:
    client = Mistral(api_key=api_key)

    # List of document URLs
    document_urls = [
        "https://xdcs.cdnchinhphu.vn/446259493575335936/2025/8/23/1-17559030806841930288416.png",
        "https://xdcs.cdnchinhphu.vn/446259493575335936/2025/8/23/2-17559030808431869401388.png",
        "https://xdcs.cdnchinhphu.vn/446259493575335936/2025/8/23/3-17559030809652084977413.png"
    ]
    
    # Store all results
    all_ocr_results = []
    combined_markdown = "# Điểm chuẩn Trường Đại học Sư phạm Kỹ thuật TP.HCM (HCMUTE) năm 2025\n\n"
    
    # Process each document individually
    for i, url in enumerate(document_urls, 1):
        print(f"Processing document {i}/{len(document_urls)}: {url}")
        
        try:
            ocr_response = client.ocr.process(
                model="mistral-ocr-latest",
                document={
                    "type": "document_url",
                    "document_url": url
                },
                include_image_base64=True
            )
            
            print(f"✓ Successfully processed document {i}")
            
            # Store result
            all_ocr_results.append({
                'document_number': i,
                'url': url,
                'response': ocr_response
            })
            
            # Initialize content variable for each document
            content = ""
            
            # Extract text content from pages
            if hasattr(ocr_response, 'pages') and ocr_response.pages:
                for page in ocr_response.pages:
                    if hasattr(page, 'markdown'):
                        content += f"{page.markdown}\n\n"
                    else:
                        content += f"{str(page)}\n\n"
            else:
                content = str(ocr_response)
            
            # Add document header and content to combined markdown
            combined_markdown += f"## Tài liệu {i}\n\n"
            combined_markdown += f"**Nguồn:** {url}\n\n"
            combined_markdown += f"{content}\n\n"
            combined_markdown += "---\n\n"

        except Exception as e:
            print(f"✗ Error processing document {i}: {e}")
            all_ocr_results.append({
                'document_number': i,
                'url': url,
                'error': str(e)
            })
    
    
    print("\n" + "="*60)
    print("COMBINED RESULTS:")
    print("="*60)
    print(combined_markdown)
    
    # Also print individual results for debugging
    print("\n" + "="*60)
    print("INDIVIDUAL OCR RESPONSES:")
    print("="*60)
    for result in all_ocr_results:
        print(f"\nDocument {result['document_number']}:")
        if 'error' in result:
            print(f"Error: {result['error']}")
        else:
            print(f"Response structure: {type(result['response'])}")
            if hasattr(result['response'], 'pages'):
                print(f"Number of pages: {len(result['response'].pages)}")

Processing document 1/3: https://xdcs.cdnchinhphu.vn/446259493575335936/2025/8/23/1-17559030806841930288416.png
✓ Successfully processed document 1
Processing document 2/3: https://xdcs.cdnchinhphu.vn/446259493575335936/2025/8/23/2-17559030808431869401388.png
✓ Successfully processed document 2
Processing document 3/3: https://xdcs.cdnchinhphu.vn/446259493575335936/2025/8/23/3-17559030809652084977413.png
✓ Successfully processed document 3

COMBINED RESULTS:
# Điểm chuẩn Trường Đại học Sư phạm Kỹ thuật TP.HCM (HCMUTE) năm 2025

## Tài liệu 1

**Nguồn:** https://xdcs.cdnchinhphu.vn/446259493575335936/2025/8/23/1-17559030806841930288416.png

BỘ GIÁO DỤC VÀ ĐÀO TẠO TRƯỜNG ĐẠI HỌC SƯ PHẠM KỸ THUẬT CHÁNH PHỐ HỐ CHÍ MINH

CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM Độc lập - Tự do - Hạnh phúc

ĐIỆ MỤNG TƯỢNG TUYỂN VÀO CÁC NGÀNH, CÁC CHƯƠNG TRÌNH ĐÀO TẠO TRÌNH ĐỘ ĐẠI HỌC HỆ CHÍNH QUY NĂM 2025 (Tờ: HỘ CHÍ MINH, theo Quyết định số 3108/QĐ-ĐHSPKT ban hành ngày 22/8/2025)

|  STT | MÃ NGÀNH | TẾN NGÀNH/CH

In [56]:
def parse_markdown_to_dataframe(markdown_content):
    """Convert markdown tables to a single DataFrame."""
    
    # Split content by document sections
    sections = markdown_content.split('## Tài liệu')
    
    all_data = []
    
    for section in sections[1:]:  # Skip the first split (header)
        # Find table content between | characters
        lines = section.split('\n')
        
        # Find lines that contain table data (start with |)
        table_lines = []
        in_table = False
        
        for line in lines:
            line = line.strip()
            if line.startswith('|') and '---' not in line:
                # This is a table row
                if 'STT' in line and 'MÃ NGÀNH' in line:
                    # This is header, skip it
                    in_table = True
                    continue
                elif in_table and line.startswith('|'):
                    table_lines.append(line)
                elif not line.startswith('|'):
                    # End of table
                    in_table = False
        
        # Parse each table line
        for line in table_lines:
            # Split by | and clean up
            cells = [cell.strip() for cell in line.split('|') if cell.strip()]
            
            if len(cells) >= 4:  # Should have STT, MÃ NGÀNH, TÊN NGÀNH, ĐIỂM CHUẨN
                try:
                    stt = int(cells[0])
                    ma_nganh = cells[1]
                    ten_nganh = cells[2]
                    diem_chuan = float(cells[3].replace(',', '.'))
                    
                    all_data.append({
                        'STT': stt,
                        'MÃ NGÀNH': ma_nganh,
                        'TÊN NGÀNH/CHƯƠNG TRÌNH ĐÀO TẠO': ten_nganh,
                        'ĐIỂM CHUẨN': diem_chuan
                    })
                except (ValueError, IndexError):
                    # Skip malformed rows
                    continue
    
    # Create DataFrame
    df = pd.DataFrame(all_data)
    
    # Sort by STT to ensure proper order
    df = df.sort_values('STT').reset_index(drop=True)
    
    return df

import pandas as pd
df = parse_markdown_to_dataframe(combined_markdown)

In [57]:
df.rename(columns={
    'MÃ NGÀNH': 'ma_nganh',
    'TÊN NGÀNH/CHƯƠNG TRÌNH ĐÀO TẠO': 'ten_nganh',
    'ĐIỂM CHUẨN': 'diem_chuan'
}, inplace=True)

# remove space in column ma_nganh
df['ma_nganh'] = df['ma_nganh'].str.replace(' ', '')

In [58]:
df = df.drop(columns=['STT'])

In [59]:
df

,ma_nganh,ten_nganh,diem_chuan
0,7140231V,Sư phạm tiếng Anh (đào tạo bằng tiếng Việt),29.57
1,7140246V,Sư phạm công nghệ (đào tạo bằng tiếng Việt),26.79
2,7210403V,Thiết kế đồ họa (đào tạo bằng tiếng Việt),24.60
3,7210404V,Thiết kế thời trang (đào tạo bằng tiếng Việt),22.70
4,7220201BP,Ngôn ngữ Anh (đào tạo tại phân hiệu Bình Phước),24.00
...,...,...,...
81,7580302BP,Quản lý xây dựng (đào tạo tại phân hiệu Bình P...,22.50
82,7580302V,Quản lý xây dựng (đào tạo bằng tiếng Việt),22.50
83,7810202V,Quản trị NH và DV ăn uống (đào tạo bằng tiếng ...,23.35
84,7840110V,Quản lý và vận hành hạ tầng (đào tạo bằng tiến...,22.00


In [60]:
df["nhom_nganh"] = df["ma_nganh"].str[:3]
df

,ma_nganh,ten_nganh,diem_chuan,nhom_nganh
0,7140231V,Sư phạm tiếng Anh (đào tạo bằng tiếng Việt),29.57,714
1,7140246V,Sư phạm công nghệ (đào tạo bằng tiếng Việt),26.79,714
2,7210403V,Thiết kế đồ họa (đào tạo bằng tiếng Việt),24.60,721
3,7210404V,Thiết kế thời trang (đào tạo bằng tiếng Việt),22.70,721
4,7220201BP,Ngôn ngữ Anh (đào tạo tại phân hiệu Bình Phước),24.00,722
...,...,...,...,...
81,7580302BP,Quản lý xây dựng (đào tạo tại phân hiệu Bình P...,22.50,758
82,7580302V,Quản lý xây dựng (đào tạo bằng tiếng Việt),22.50,758
83,7810202V,Quản trị NH và DV ăn uống (đào tạo bằng tiếng ...,23.35,781
84,7840110V,Quản lý và vận hành hạ tầng (đào tạo bằng tiến...,22.00,784


In [61]:
df["tp"] = "TP. Hồ Chí Minh"
df.loc[df["ma_nganh"].str.contains("BP", na=False), "tp"] = "Bình Phước"

In [62]:
df["cong_lap"] = 1

In [63]:
hoc_phi = pd.read_excel("../../data/train_l1.xlsx")
hoc_phi

,Công lập (1/0),Tỉnh/TP,Tổ hợp môn,Học phí,hsg_1,hsg_2,hsg_3,ahld,dan_toc_thieu_so,haimuoi_huyen_ngheo_tnb,truong-nganh-ptxt,nhom_nganh
0,1,An Giang,NaN,14100000.0,Toán,0,0,0,0,0,QSA-7140201-TUYỂN THẲNG,714
1,1,An Giang,NaN,14100000.0,Lý,0,0,0,0,0,QSA-7140201-TUYỂN THẲNG,714
2,1,An Giang,NaN,14100000.0,Hoá,0,0,0,0,0,QSA-7140201-TUYỂN THẲNG,714
3,1,An Giang,NaN,14100000.0,Sinh,0,0,0,0,0,QSA-7140201-TUYỂN THẲNG,714
4,1,An Giang,NaN,14100000.0,Tin,0,0,0,0,0,QSA-7140201-TUYỂN THẲNG,714
...,...,...,...,...,...,...,...,...,...,...,...,...
40910,1,TP. Hồ Chí Minh,C01,62000000.0,0,0,0,0,0,1,SPK-7510303A-Ưu Tiên,751
40911,1,Bình Phước,C01,32000000.0,0,0,0,0,0,1,SPK-7510301BP-Ưu Tiên,751
40912,1,TP. Hồ Chí Minh,C02,32000000.0,0,0,0,0,0,1,SPK-7510401V-Ưu Tiên,751
40913,1,TP. Hồ Chí Minh,A01,49000000.0,0,0,0,0,0,1,SPK-7510205N-Ưu Tiên,751


In [64]:
hoc_phi = hoc_phi[hoc_phi['truong-nganh-ptxt'].str.contains("SPK")]
hoc_phi

,Công lập (1/0),Tỉnh/TP,Tổ hợp môn,Học phí,hsg_1,hsg_2,hsg_3,ahld,dan_toc_thieu_so,haimuoi_huyen_ngheo_tnb,truong-nganh-ptxt,nhom_nganh
11471,1,TP. Hồ Chí Minh,C01,36000000.0,Toán,0,0,0,0,0,SPK-7340120V-Ưu Tiên,734
11472,1,TP. Hồ Chí Minh,C01,36000000.0,Văn,0,0,0,0,0,SPK-7340120V-Ưu Tiên,734
11473,1,TP. Hồ Chí Minh,C01,36000000.0,Lý,0,0,0,0,0,SPK-7340120V-Ưu Tiên,734
11474,1,TP. Hồ Chí Minh,A01,36000000.0,Toán,0,0,0,0,0,SPK-7340120V-Ưu Tiên,734
11475,1,TP. Hồ Chí Minh,A01,36000000.0,Lý,0,0,0,0,0,SPK-7340120V-Ưu Tiên,734
...,...,...,...,...,...,...,...,...,...,...,...,...
40910,1,TP. Hồ Chí Minh,C01,62000000.0,0,0,0,0,0,1,SPK-7510303A-Ưu Tiên,751
40911,1,Bình Phước,C01,32000000.0,0,0,0,0,0,1,SPK-7510301BP-Ưu Tiên,751
40912,1,TP. Hồ Chí Minh,C02,32000000.0,0,0,0,0,0,1,SPK-7510401V-Ưu Tiên,751
40913,1,TP. Hồ Chí Minh,A01,49000000.0,0,0,0,0,0,1,SPK-7510205N-Ưu Tiên,751


In [65]:
hoc_phi = hoc_phi.drop_duplicates("truong-nganh-ptxt")

In [66]:
hoc_phi = hoc_phi[['truong-nganh-ptxt', 'Học phí']]

In [67]:
hoc_phi["nganh"] = hoc_phi["truong-nganh-ptxt"].str.extract(r'SPK-([^-]+)-')

In [68]:
hoc_phi = hoc_phi.drop(columns=['truong-nganh-ptxt'])

In [69]:
# Merge df with hoc_phi on nganh
df = df.merge(hoc_phi, left_on='ma_nganh', right_on='nganh', how='left')
df = df.drop(columns=['nganh'])

In [70]:
df

,ma_nganh,ten_nganh,diem_chuan,nhom_nganh,tp,cong_lap,Học phí
0,7140231V,Sư phạm tiếng Anh (đào tạo bằng tiếng Việt),29.57,714,TP. Hồ Chí Minh,1,30000000.0
1,7140246V,Sư phạm công nghệ (đào tạo bằng tiếng Việt),26.79,714,TP. Hồ Chí Minh,1,30000000.0
2,7210403V,Thiết kế đồ họa (đào tạo bằng tiếng Việt),24.60,721,TP. Hồ Chí Minh,1,30000000.0
3,7210404V,Thiết kế thời trang (đào tạo bằng tiếng Việt),22.70,721,TP. Hồ Chí Minh,1,30000000.0
4,7220201BP,Ngôn ngữ Anh (đào tạo tại phân hiệu Bình Phước),24.00,722,Bình Phước,1,30000000.0
...,...,...,...,...,...,...,...
81,7580302BP,Quản lý xây dựng (đào tạo tại phân hiệu Bình P...,22.50,758,Bình Phước,1,32000000.0
82,7580302V,Quản lý xây dựng (đào tạo bằng tiếng Việt),22.50,758,TP. Hồ Chí Minh,1,32000000.0
83,7810202V,Quản trị NH và DV ăn uống (đào tạo bằng tiếng ...,23.35,781,TP. Hồ Chí Minh,1,30000000.0
84,7840110V,Quản lý và vận hành hạ tầng (đào tạo bằng tiến...,22.00,784,TP. Hồ Chí Minh,1,32000000.0


In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ma_nganh    86 non-null     object 
 1   ten_nganh   86 non-null     object 
 2   diem_chuan  86 non-null     float64
 3   nhom_nganh  86 non-null     object 
 4   tp          86 non-null     object 
 5   cong_lap    86 non-null     int64  
 6   Học phí     82 non-null     float64
dtypes: float64(2), int64(1), object(4)
memory usage: 4.8+ KB


In [74]:
# print row có học phí null
print(df[df['Học phí'].isnull()])

       ma_nganh                                          ten_nganh  \
52  7510302KTVM  Chương trình đào tạo Kỹ thuật Thiết kế vi mạch...   

    diem_chuan nhom_nganh               tp  cong_lap  Học phí  
52       28.65        751  TP. Hồ Chí Minh         1      NaN  


In [77]:
# trong cột ma_nganh, thay thế KTVM thành TKVM
df['ma_nganh'] = df['ma_nganh'].str.replace('KTVM', 'TKVM')

# Set học phí của mã ngành 7510201TDA, 7510401A, 7510402A là 62000000, 7510302TKVM là 38000000
df.loc[df['ma_nganh'] == '7510201TDA', 'Học phí'] = 62000000
df.loc[df['ma_nganh'] == '7510401A', 'Học phí'] = 62000000
df.loc[df['ma_nganh'] == '7510402A', 'Học phí'] = 62000000
df.loc[df['ma_nganh'] == '7510302TKVM', 'Học phí'] = 38000000
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ma_nganh    86 non-null     object 
 1   ten_nganh   86 non-null     object 
 2   diem_chuan  86 non-null     float64
 3   nhom_nganh  86 non-null     object 
 4   tp          86 non-null     object 
 5   cong_lap    86 non-null     int64  
 6   Học phí     86 non-null     float64
dtypes: float64(2), int64(1), object(4)
memory usage: 4.8+ KB


In [79]:
df.to_excel("../../data/hocba_l3.xlsx", index=False)